In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23828602
paper_name = 'ding_bakalinsky_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits_genenames_R.txt', header=None, names=['genes'], sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 31 x 1


In [7]:
original_data.head()

,genes
0,EDE1
1,FTH1
2,MVB12
3,PRK1
4,VAM7


In [8]:
original_data['genes'] = original_data['genes'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, orf]
Index: []


In [12]:
original_data['data'] = 1

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(22, 1)

# Load & process tested strains

In [19]:
tested = pd.read_excel('raw_data/YSC1054Y.copy.xlsx', sheet_name='mat_alpha_obs')

In [20]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,notes
0,10405.0,YAL002W,BY4739,01_1,101.0,E,1,NaN
1,10402.0,YAL004W,BY4739,01_1,101.0,D,11,NaN
2,10403.0,YAL005C,BY4739,01_1,101.0,D,12,NaN
3,10401.0,YAL007C,BY4739,01_1,101.0,D,10,NaN
4,10400.0,YAL008W,BY4739,01_1,101.0,D,9,NaN


In [21]:
tested['orf'] = tested['ORF name'].astype(str)

In [22]:
tested['orf'] = clean_orf(tested['orf'])

In [23]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [24]:
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch  plate  row   column notes  orf
index_input                                                                  
4826                NaN      NaN    NaN   NaN    NaN  NaN  end 150   NaN  NAN


In [25]:
tested = tested.loc[t,:]

In [26]:
tested_orfs = tested['orf'].unique()

In [27]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [28]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [144]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,144
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,144
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        144       
data_type       value valuez
gene_id orf                 
2       YAL002W     0    0.0
1863    YAL004W     0    0.0
4       YAL005C     0    0.0
5       YAL007C     0    0.0
6       YAL008W     0    0.0

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23828602...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.39s/it]

Updating the data_modified_on field...
